In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

#### Load GloVe embeddings

In [2]:
embeddings_index = {}

f = open ('glove.6B.100d.txt','r', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word]=coefs

f.close()
print ('Found %s word embeddings'%(len(embeddings_index)))

Found 400000 word embeddings


### Load comments

In [3]:
input_texts = []
target_texts = []

for line in open('./data/poetry/robert_frost.txt'):
    line = line.rstrip()
    input_line = '<sos>'+line
    target_line = line+'<eos>'
    input_texts.append(input_line)
    target_texts.append(target_line)
    
all_lines = input_texts + target_texts

In [4]:
MAX_WORDS=20000
BATCH_SIZE=16
EPOCHS=500
OOV_TOKEN=0
EMBEDDING_DIM = 100
MAX_SEQ_LENGTH=100
VALIDATION_SPLIT_RATIO= 0.3
LSTM_UNITS=64

### Tokenize comments

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS, oov_token=OOV_TOKEN, filters='')
tokenizer.fit_on_texts(all_lines)
word2Idx = tokenizer.word_index

print ('Found %d unique words'%(len(word2Idx)))
print (word2Idx['girl'])

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
print ('Found %d input sequences'%(len(input_sequences)))
print ('Found %d output sequences'%(len(target_sequences)))

max_seq_len_from_data = min (MAX_SEQ_LENGTH, max(len(s) for s in input_sequences))

padded_input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, padding='post', maxlen=max_seq_len_from_data)
padded_target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, padding='post', maxlen=max_seq_len_from_data)

print ('Created %d padded input sequences'%(len(padded_input_sequences)))
print ('Created %d padded target sequences'%(len(padded_target_sequences)))

assert ('<sos>' in word2Idx)
assert ('<eos>' in word2Idx)

Found 4617 unique words
1268
Found 1581 input sequences
Found 1581 output sequences
Created 1581 padded input sequences
Created 1581 padded target sequences


In [6]:
num_words = min (MAX_WORDS, len(word2Idx)+1)
print ('Min words to be considered are %d'%(num_words))

loaded_embeddings_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word2Idx.items():
    if (i<num_words):
        if word in embeddings_index.keys():
            embedding_vector = embeddings_index[word]
            loaded_embeddings_matrix[i] = embedding_vector

print (loaded_embeddings_matrix.shape)

Min words to be considered are 4618
(4618, 100)


In [7]:
for i, seq in enumerate(padded_target_sequences):
    print (seq)
    if (i>2):
        break

[ 136  574  575    8    4  576 3470    0    0    0    0]
[   5  577    6   69   31  987 3471    0    0    0    0]
[   5   27   28  988  166    6 3472    0    0    0    0]
[   5  184   72   28   17  140   17    6 3473    0    0]


### Important Step:

If we look at this problem statement, we have a set of input sequences and corresponding target sequences.

Input Seq (T*D)-> Target Seq (T'*D')

    T -> number of input sequences
    D -> length of each input sequence

    T' -> number of target sequences
    D' -> length of each target sequence.

In a seq2seq scenario the target value needs to be one-hot encoded and that's what we are doing below.

In [8]:
one_hot_targets = np.zeros((len(target_sequences), max_seq_len_from_data, num_words))
print (one_hot_targets.shape)
for i, seq in enumerate(padded_target_sequences):
    for j, word in enumerate(seq):
        if (word>0):
            one_hot_targets[i,j,word]=1

(1581, 11, 4618)


### Build Model

In [9]:
embed_layer = tf.keras.layers.Embedding(input_dim = num_words,
                               output_dim = EMBEDDING_DIM,
                               input_length=max_seq_len_from_data, 
                               embeddings_initializer=tf.keras.initializers.Constant(loaded_embeddings_matrix),
                               trainable=False)

In [10]:
input_ = tf.keras.layers.Input(shape=(max_seq_len_from_data))
initial_h = tf.keras.layers.Input(shape=(LSTM_UNITS,))
initial_c = tf.keras.layers.Input(shape=(LSTM_UNITS,))

x = embed_layer(input_)

lstm_layer_0 = tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, return_state=True)
x,_,_ = lstm_layer_0(x)

lstm_layer_1 = tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, return_state=True)
x,_,_ = lstm_layer_1(x)

dense_layer = tf.keras.layers.Dense(num_words, activation='softmax')
output = dense_layer(x)

model = tf.keras.models.Model([input_, initial_h, initial_c], output)

model.compile (optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_targets, test_targets = train_test_split (padded_input_sequences, one_hot_targets)

In [12]:
h = np.zeros((len(train_sentences),LSTM_UNITS))
c = np.zeros((len(train_sentences),LSTM_UNITS))
model.fit ([train_sentences,h,c], train_targets, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT_RATIO)

W0825 14:34:17.527918 14468 deprecation.py:323] From C:\MachineLearning\anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 829 samples, validate on 356 samples
Epoch 1/500
829/829 [==============================] - 7s 9ms/sample - loss: 5.0030 - accuracy: 0.0263 - val_loss: 4.5988 - val_accuracy: 0.0324
Epoch 2/500
829/829 [==============================] - 4s 5ms/sample - loss: 4.3728 - accuracy: 0.0306 - val_loss: 4.7203 - val_accuracy: 0.0324
Epoch 3/500
829/829 [==============================] - 4s 5ms/sample - loss: 4.2404 - accuracy: 0.0306 - val_loss: 4.5039 - val_accuracy: 0.0324
Epoch 4/500
829/829 [==============================] - 4s 5ms/sample - loss: 4.0833 - accuracy: 0.0306 - val_loss: 4.3944 - val_accuracy: 0.0324
Epoch 5/500
829/829 [==============================] - 4s 5ms/sample - loss: 3.9775 - accuracy: 0.0306 - val_loss: 4.3628 - val_accuracy: 0.0324
Epoch 6/500
829/829 [==============================] - 4s 5ms/sample - loss: 3.8921 - accuracy: 0.0306 - val_loss: 4.3440 - val_accuracy: 0.0324
Epoch 7/500
829/829 [==============================] - 4s 5ms/sample - loss: 3.8195 

Epoch 57/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.7964 - accuracy: 0.3118 - val_loss: 3.8218 - val_accuracy: 0.2400
Epoch 58/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.7640 - accuracy: 0.3213 - val_loss: 3.8287 - val_accuracy: 0.2408
Epoch 59/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.7324 - accuracy: 0.3278 - val_loss: 3.8132 - val_accuracy: 0.2444
Epoch 60/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.7005 - accuracy: 0.3380 - val_loss: 3.8141 - val_accuracy: 0.2446
Epoch 61/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.6699 - accuracy: 0.3447 - val_loss: 3.8038 - val_accuracy: 0.2503
Epoch 62/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.6393 - accuracy: 0.3533 - val_loss: 3.8077 - val_accuracy: 0.2505
Epoch 63/500
829/829 [==============================] - 4s 5ms/sample - loss: 1.6084 - accuracy: 0.3623 - val_loss: 3.8041 -

829/829 [==============================] - 4s 5ms/sample - loss: 0.6893 - accuracy: 0.5166 - val_loss: 3.7305 - val_accuracy: 0.2893
Epoch 114/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6828 - accuracy: 0.5160 - val_loss: 3.7313 - val_accuracy: 0.2863
Epoch 115/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6751 - accuracy: 0.5172 - val_loss: 3.7346 - val_accuracy: 0.2880
Epoch 116/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6680 - accuracy: 0.5171 - val_loss: 3.7354 - val_accuracy: 0.2888
Epoch 117/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6616 - accuracy: 0.5188 - val_loss: 3.7342 - val_accuracy: 0.2888
Epoch 118/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6546 - accuracy: 0.5189 - val_loss: 3.7336 - val_accuracy: 0.2888
Epoch 119/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.6481 - accuracy: 0.5196 - val_loss: 3.7358 - val_ac

Epoch 169/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5069 - accuracy: 0.5260 - val_loss: 3.7827 - val_accuracy: 0.2929
Epoch 170/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5058 - accuracy: 0.5257 - val_loss: 3.7858 - val_accuracy: 0.2929
Epoch 171/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5045 - accuracy: 0.5269 - val_loss: 3.7862 - val_accuracy: 0.2909
Epoch 172/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5038 - accuracy: 0.5255 - val_loss: 3.7876 - val_accuracy: 0.2901
Epoch 173/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5033 - accuracy: 0.5274 - val_loss: 3.7889 - val_accuracy: 0.2926
Epoch 174/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5023 - accuracy: 0.5273 - val_loss: 3.7868 - val_accuracy: 0.2911
Epoch 175/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5017 - accuracy: 0.5257 - val_loss: 3

Epoch 225/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4757 - accuracy: 0.5265 - val_loss: 3.8285 - val_accuracy: 0.2921
Epoch 226/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4753 - accuracy: 0.5266 - val_loss: 3.8308 - val_accuracy: 0.2914
Epoch 227/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4753 - accuracy: 0.5270 - val_loss: 3.8305 - val_accuracy: 0.2937
Epoch 228/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4751 - accuracy: 0.5258 - val_loss: 3.8318 - val_accuracy: 0.2921
Epoch 229/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4751 - accuracy: 0.5265 - val_loss: 3.8329 - val_accuracy: 0.2939
Epoch 230/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4750 - accuracy: 0.5264 - val_loss: 3.8342 - val_accuracy: 0.2937
Epoch 231/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4743 - accuracy: 0.5266 - val_loss: 3

Epoch 281/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4683 - accuracy: 0.5271 - val_loss: 3.8588 - val_accuracy: 0.2924
Epoch 282/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4685 - accuracy: 0.5245 - val_loss: 3.8595 - val_accuracy: 0.2921
Epoch 283/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4683 - accuracy: 0.5262 - val_loss: 3.8601 - val_accuracy: 0.2919
Epoch 284/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4683 - accuracy: 0.5265 - val_loss: 3.8604 - val_accuracy: 0.2944
Epoch 285/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4685 - accuracy: 0.5263 - val_loss: 3.8616 - val_accuracy: 0.2947
Epoch 286/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4684 - accuracy: 0.5260 - val_loss: 3.8615 - val_accuracy: 0.2944
Epoch 287/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4682 - accuracy: 0.5255 - val_loss: 3

Epoch 337/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4664 - accuracy: 0.5264 - val_loss: 3.8816 - val_accuracy: 0.2937
Epoch 338/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4659 - accuracy: 0.5266 - val_loss: 3.8837 - val_accuracy: 0.2921
Epoch 339/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4662 - accuracy: 0.5269 - val_loss: 3.8848 - val_accuracy: 0.2944
Epoch 340/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4660 - accuracy: 0.5263 - val_loss: 3.8857 - val_accuracy: 0.2947
Epoch 341/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4660 - accuracy: 0.5255 - val_loss: 3.8848 - val_accuracy: 0.2944
Epoch 342/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4758 - accuracy: 0.5246 - val_loss: 3.9162 - val_accuracy: 0.2845
Epoch 343/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.5094 - accuracy: 0.5173 - val_loss: 3

Epoch 393/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4653 - accuracy: 0.5256 - val_loss: 3.8950 - val_accuracy: 0.2932
Epoch 394/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4652 - accuracy: 0.5264 - val_loss: 3.8950 - val_accuracy: 0.2955
Epoch 395/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4652 - accuracy: 0.5267 - val_loss: 3.8958 - val_accuracy: 0.2934
Epoch 396/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4649 - accuracy: 0.5264 - val_loss: 3.8960 - val_accuracy: 0.2939
Epoch 397/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4653 - accuracy: 0.5252 - val_loss: 3.8959 - val_accuracy: 0.2955
Epoch 398/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4652 - accuracy: 0.5263 - val_loss: 3.8965 - val_accuracy: 0.2962
Epoch 399/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4651 - accuracy: 0.5259 - val_loss: 3

Epoch 449/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4642 - accuracy: 0.5260 - val_loss: 3.9099 - val_accuracy: 0.2939
Epoch 450/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4643 - accuracy: 0.5270 - val_loss: 3.9105 - val_accuracy: 0.2947
Epoch 451/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4646 - accuracy: 0.5267 - val_loss: 3.9119 - val_accuracy: 0.2924
Epoch 452/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4644 - accuracy: 0.5263 - val_loss: 3.9112 - val_accuracy: 0.2947
Epoch 453/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4644 - accuracy: 0.5273 - val_loss: 3.9120 - val_accuracy: 0.2949
Epoch 454/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4644 - accuracy: 0.5266 - val_loss: 3.9122 - val_accuracy: 0.2952
Epoch 455/500
829/829 [==============================] - 4s 5ms/sample - loss: 0.4644 - accuracy: 0.5260 - val_loss: 3

### Prediction Model

In [13]:
input2_ = tf.keras.layers.Input(shape=(1,))

x = embed_layer(input2_)
x,h,c = lstm_layer_0(x)
x,h,c = lstm_layer_1(x)

output2_ = dense_layer(x)

pred_model = tf.keras.models.Model([input2_, initial_h, initial_c], [output2_,h,c])
pred_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             461800      input_4[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     multiple             42240       embedding[1][0]                  
__________________________________________________________________________________________________
lstm_1 (LSTM)                   multiple             33024       lstm[1][0]                       
____________________________________________________________________________________________

In [14]:
test_input_word = 'both'
test_input_word_idx = word2Idx[test_input_word]
test_input_word_vector = np.array([[test_input_word_idx]])

print (test_input_word_vector.shape)

h = np.zeros([1,LSTM_UNITS])
c = np.zeros([1,LSTM_UNITS])

output_poem = []

idx2word = {v:k for k, v in word2Idx.items()}

for i in range(max_seq_len_from_data):
    
    print ('Start word is ',test_input_word_vector)
    o,h,c = pred_model.predict([test_input_word_vector, h, c])
    
    prob_word = np.argmax(o)    
    pred_word = idx2word[prob_word]
    
    if (pred_word == '<eos>'):
        break
    
    test_input_word_vector[0,0] = prob_word
    
    output_poem.append(pred_word + ' ')

(1, 1)
Start word is  [[156]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]
Start word is  [[18]]


In [15]:
print (output_poem)

['on ', 'on ', 'on ', 'on ', 'on ', 'on ', 'on ', 'on ', 'on ', 'on ', 'on ']
